In [202]:
import cv2
import itertools as it
import os
import sys
import multiprocessing as mp
import glob
import pandas as pd
import numpy as np

In [203]:
def splitImagesByAspectRatio(path):
    for folder in os.listdir(path):
        for file in os.listdir(path + folder):
            img = cv2.imread(path + folder + "/" + file)
            height, width, channels = img.shape
            aspect_ratio = width/height
            createSquaredImages(img, aspect_ratio, path,folder,file)

def createSquaredImages(img, aspect_ratio, path,folder,file):
    if(aspect_ratio == 4/3):
        square_shape = img.shape[0] // 6
    else:
        square_shape = img.shape[0] // 18
    count = 0
    if not os.path.exists(os.path.join(path,folder)):
        os.mkdir(os.path.join(path,folder))
    for i in np.arange(0,img.shape[0],square_shape):
        for j in np.arange(0, img.shape[1],square_shape):
            subfig = img[i:i+square_shape,j:j+square_shape]
            if j >= img.shape[1]/2:
                subfig = cv2.flip(subfig,1)
            figname = os.path.join(path, folder) + "/" + file.split('.')[0] + '-' + str(count) + '.' + file.split('.')[1]
            print(figname)
            cv2.imwrite(figname, subfig)
            count += 1               

In [204]:
path = '/media/newdrive/'
new_folder = os.path.join(path, 'processed_sss')
print(new_folder)
if not os.path.exists(new_folder):
    os.mkdir(new_folder)
count = 0
images = []
for img in glob.iglob(path + '**/*.png', recursive=True):
    count += 1
    print(count, img)
    images.append(img)

/media/newdrive/processed_sss
1 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-4.png
2 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-10.png
3 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-15.png
4 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-19.png
5 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-18.png
6 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-9.png
7 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-8.png
8 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-20.png
9 /media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/105036_np3_cam-16.png
10 /media/newdrive/lauv-noptilus-3/20

In [205]:
folders = []
dfs = []
for img in images:
    f = '/'.join(img.split('/')[:-3])
    if(f not in folders):
        if os.path.exists(os.path.join(f,'mra/filtered_sss_images/positions.csv')):
            print(f)
            print(os.path.join(f,'mra/filtered_sss_images/positions.csv'))
            dfs.append(os.path.join(f,'mra/filtered_sss_images/positions.csv'))
            folders.append(f)


/media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam
/media/newdrive/lauv-noptilus-3/20180711/105036_np3_cam/mra/filtered_sss_images/positions.csv
/media/newdrive/lauv-noptilus-3/20180711/115633_np3_cam2
/media/newdrive/lauv-noptilus-3/20180711/115633_np3_cam2/mra/filtered_sss_images/positions.csv
/media/newdrive/lauv-noptilus-3/20180710/100154_A003_NP3
/media/newdrive/lauv-noptilus-3/20180710/100154_A003_NP3/mra/filtered_sss_images/positions.csv
/media/newdrive/lauv-noptilus-3/20180710/133936_cam_np3_2
/media/newdrive/lauv-noptilus-3/20180710/133936_cam_np3_2/mra/filtered_sss_images/positions.csv
/media/newdrive/lauv-noptilus-3/20180710/122322_cam_np3
/media/newdrive/lauv-noptilus-3/20180710/122322_cam_np3/mra/filtered_sss_images/positions.csv
/media/newdrive/lauv-noptilus-3/20180501/113610_cam_survey_1
/media/newdrive/lauv-noptilus-3/20180501/113610_cam_survey_1/mra/filtered_sss_images/positions.csv
/media/newdrive/lauv-noptilus-3/20180501/104728_cam-np3
/media/newdrive/lauv-nopti

In [206]:
len(dfs)

24

In [207]:
df = pd.DataFrame(columns=['filename','timestamp','latitude','longitude','distance','depth','altitude','roll','pitch',
                           'auv','date'])

i = 0
for d in dfs:
    ps = d.split('/')
    auv = ps[3]
    date = ps[4][-2:] + '/' + ps[4][-4:-2] + '/' + ps[4][:4]
    log_df = pd.read_csv(d)
    for index, row in log_df.iterrows():
        df.loc[i] = [row['filename'] + '-' + str(i%12), row['timestamp'], row['latitude'], row['longitude'], 
                   row['distance'], row['depth'], row['altitude'], row['roll'], row['pitch'], auv, date]
        i += 1

In [208]:
print(i , len(df))


9856 9856


In [209]:
df.to_csv(os.path.join(new_folder, 'df.csv'), index=False)
#df = pd.read_csv(os.path.join(new_folder, 'df.csv'))

In [210]:
for imgname in images:
    img = cv2.imread(imgname)
    ap = 4/3
    folder = imgname.split('/')[5]
    createSquaredImages(img, ap, new_folder, folder, imgname.split('/')[-1])
    

/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-0.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-1.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-2.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-3.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-4.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-5.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-6.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-7.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-8.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-9.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-10.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-11.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-12.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-4-13.png
/media/newdrive/processed_sss/105036_np3_cam

/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-19-22.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-19-23.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-19-24.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-19-25.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-19-26.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-19-27.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-19-28.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-19-29.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-19-30.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-19-31.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-19-32.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-19-33.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-19-34.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-19-35.png
/media/newdrive/proc

/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-20-0.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-20-1.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-20-2.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-20-3.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-20-4.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-20-5.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-20-6.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-20-7.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-20-8.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-20-9.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-20-10.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-20-11.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-20-12.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-20-13.png
/media/newdrive/processed_sss/

/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-5-30.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-5-31.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-5-32.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-5-33.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-5-34.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-5-35.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-5-36.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-5-37.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-5-38.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-5-39.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-5-40.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-5-41.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-5-42.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-5-43.png
/media/newdrive/processed_sss/1050

/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-3-34.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-3-35.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-3-36.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-3-37.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-3-38.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-3-39.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-3-40.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-3-41.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-3-42.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-3-43.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-3-44.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-3-45.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-3-46.png
/media/newdrive/processed_sss/105036_np3_cam/105036_np3_cam-3-47.png
/media/newdrive/processed_sss/1050

...

/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-69-0.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-69-1.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-69-2.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-69-3.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-69-4.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-69-5.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-69-6.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-69-7.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-69-8.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-69-9.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-69-10.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-69-11.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-69-12.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-69-13.png
/m

/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-21-0.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-21-1.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-21-2.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-21-3.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-21-4.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-21-5.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-21-6.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-21-7.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-21-8.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-21-9.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-21-10.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-21-11.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-21-12.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-21-13.png
/m

/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-26-4.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-26-5.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-26-6.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-26-7.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-26-8.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-26-9.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-26-10.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-26-11.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-26-12.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-26-13.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-26-14.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-26-15.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-26-16.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-26-17.pn

/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-46-0.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-46-1.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-46-2.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-46-3.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-46-4.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-46-5.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-46-6.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-46-7.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-46-8.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-46-9.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-46-10.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-46-11.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-46-12.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-46-13.png
/m

/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-10-0.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-10-1.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-10-2.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-10-3.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-10-4.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-10-5.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-10-6.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-10-7.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-10-8.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-10-9.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-10-10.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-10-11.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-10-12.png
/media/newdrive/processed_sss/123158_A032_NP2/123158_A032_NP2-10-13.png
/m

In [211]:
from math import sin, cos, sqrt, atan2, radians

def getGreatCircleDistance(lat1,lon1,lat2,lon2):
    R = 6371.009
    
    lat1 = radians(abs(lat1))
    lon1 = radians(abs(lon1))
    lat2 = radians(abs(lat2))
    lon2 = radians(abs(lon2))
    
    delta_lon = lon2 - lon1
    delta_lat = lat2 - lat1

    a = sin(delta_lat / 2)**2 + cos(lat1) * cos(lat2) * sin(delta_lon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
print(getGreatCircleDistance(41.53283, -8.81532, 41.53101, -8.80974))

In [15]:
labeled_data = pd.read_csv('/home/diegues/Desktop/ProcessedImages/labeled_data.csv')
labeled_data.head()
        

,filename,timestamp,latitude,longitude,roll,pitch,entropy,date,depth,EunisCode,EunisName,level1,level2,level3,level4,level5,level6,species,AphiaID
0,104728_cam-np3_frame5.jpg,1.525172e+09,41.53283,-8.81532,-3.36,-14.12,21.934646,01/05/18,16.97,A4.1,Atlantic and Mediterranean high energy circali...,A,A4,A4.1,NaN,NaN,NaN,Mesophyllum lichenoides,145188.0
1,104728_cam-np3_frame9.jpg,1.525172e+09,41.53283,-8.81531,-2.34,-16.76,21.932552,01/05/18,16.95,A4.1,Atlantic and Mediterranean high energy circali...,A,A4,A4.1,NaN,NaN,NaN,Mesophyllum lichenoides,145188.0
2,104728_cam-np3_frame50.jpg,1.525172e+09,41.53277,-8.81526,-25.75,-17.73,21.933617,01/05/18,16.11,A4.1,Atlantic and Mediterranean high energy circali...,A,A4,A4.1,NaN,NaN,NaN,Mesophyllum lichenoides,145188.0
3,104728_cam-np3_frame51.jpg,1.525172e+09,41.53277,-8.81526,-25.75,-17.73,21.934235,01/05/18,16.11,A4.1,Atlantic and Mediterranean high energy circali...,A,A4,A4.1,NaN,NaN,NaN,Mesophyllum lichenoides,145188.0
4,104728_cam-np3_frame52.jpg,1.525172e+09,41.53277,-8.81526,-25.75,-17.73,21.909359,01/05/18,16.11,A4.1,Atlantic and Mediterranean high energy circali...,A,A4,A4.1,NaN,NaN,NaN,Echinus esculentus,124287.0


In [18]:
level3 = {}
imagedata = []
for index, row in labeled_data.iterrows():
    for index2, row2 in df.iterrows():
        distance = getGreatCircleDistance(row['latitude'], row['longitude'], row2['latitude'], row2['longitude'])
        #print(row['filename'],row2['filename'],distance)
        if distance <= 0.005:
            level3[row2['filename']] = row['level3']
            imagedata.append(row['filename'] + ',' + row2['filename'] + ',' + row['level3'] + ',' + str(distance))

KeyboardInterrupt: 

In [ ]:
len(level3)


In [ ]:
len(imagedata)

In [ ]:
lcl_df = pd.DataFrame(columns=['photo_name', 'sss_name', 'level3', 'distance'])

In [ ]:
index = 0
for row in imagedata:
    lcl_df.loc[index] = row.split(',')
    index += 1
    

In [ ]:
lcl_df.head()

In [41]:
#lcl_df.to_csv(os.path.join(new_folder,'local_match.csv'),index=False)
lcl_df = pd.read_csv(os.path.join(new_folder,'local_match.csv'))

In [42]:
lcl_df.head()

,photo_name,sss_name,level3,distance
0,104728_cam-np3_frame5.jpg,105317_cam-np3-1-9,A4.1,0.007249
1,104728_cam-np3_frame5.jpg,105317_cam-np3-1-10,A4.1,0.006709
2,104728_cam-np3_frame5.jpg,105317_cam-np3-2-11,A4.1,0.001584
3,104728_cam-np3_frame5.jpg,105317_cam-np3-2-2,A4.1,0.004862
4,104728_cam-np3_frame5.jpg,112045_mb_survey-1-0,A4.1,0.009011


In [43]:
# escolher a class mais proxima do ponto central 
new_levels = pd.get_dummies(lcl_df.level3)
new_levels.head()

,A3.1,A3.7,A4.1,A4.7,A5.1,A5.2,A5.4
0,0,0,1,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0
4,0,0,1,0,0,0,0


In [44]:
#lcl_df = lcl_df.drop('level3', axis=1)
lcl_df = pd.concat([lcl_df, new_levels], axis=1)
lcl_df.head()

,photo_name,sss_name,level3,distance,A3.1,A3.7,A4.1,A4.7,A5.1,A5.2,A5.4
0,104728_cam-np3_frame5.jpg,105317_cam-np3-1-9,A4.1,0.007249,0,0,1,0,0,0,0
1,104728_cam-np3_frame5.jpg,105317_cam-np3-1-10,A4.1,0.006709,0,0,1,0,0,0,0
2,104728_cam-np3_frame5.jpg,105317_cam-np3-2-11,A4.1,0.001584,0,0,1,0,0,0,0
3,104728_cam-np3_frame5.jpg,105317_cam-np3-2-2,A4.1,0.004862,0,0,1,0,0,0,0
4,104728_cam-np3_frame5.jpg,112045_mb_survey-1-0,A4.1,0.009011,0,0,1,0,0,0,0


In [45]:
min_dist = lcl_df[['sss_name', 'level3', 'distance']].groupby('sss_name').min()
min_dist.head()

,level3,distance
sss_name,,
104728_cam-np3-1-4,A4.7,0.007185
104728_cam-np3-1-5,A4.1,0.007478
104728_cam-np3-2-10,A4.1,0.007234
104728_cam-np3-2-8,A4.7,0.008970
104728_cam-np3-2-9,A4.1,0.007015


In [46]:
sss_classes = lcl_df.drop(['photo_name', 'level3', 'distance'], axis = 1).groupby('sss_name').sum()
sss_classes.head()

,A3.1,A3.7,A4.1,A4.7,A5.1,A5.2,A5.4
sss_name,,,,,,,
104728_cam-np3-1-4,0,0,0,57,0,0,0
104728_cam-np3-1-5,0,0,3,29,0,0,0
104728_cam-np3-2-10,0,0,26,0,0,0,0
104728_cam-np3-2-8,0,0,0,57,0,0,0
104728_cam-np3-2-9,0,0,26,52,0,0,0


In [47]:
sss_df = pd.concat([min_dist, sss_classes], axis = 1)
sss_df

,level3,distance,A3.1,A3.7,A4.1,A4.7,A5.1,A5.2,A5.4
sss_name,,,,,,,,,
104728_cam-np3-1-4,A4.7,0.007185,0,0,0,57,0,0,0
104728_cam-np3-1-5,A4.1,0.007478,0,0,3,29,0,0,0
104728_cam-np3-2-10,A4.1,0.007234,0,0,26,0,0,0,0
104728_cam-np3-2-8,A4.7,0.008970,0,0,0,57,0,0,0
104728_cam-np3-2-9,A4.1,0.007015,0,0,26,52,0,0,0
105317_cam-np3-1-10,A4.1,0.005886,0,0,28,52,0,0,0
105317_cam-np3-1-11,A4.7,0.005251,0,0,0,57,0,0,0
105317_cam-np3-1-2,A4.1,0.005842,0,0,3,29,0,0,0
105317_cam-np3-1-3,A4.7,0.004719,0,0,0,57,0,0,0


In [53]:
sss_df.columns = ['min_distance_class', 'min_distance'] + list(sss_df.columns[2:])
sss_df.head()

,min_distance_class,min_distance,A3.1,A3.7,A4.1,A4.7,A5.1,A5.2,A5.4
sss_name,,,,,,,,,
104728_cam-np3-1-4,A4.7,0.007185,0,0,0,57,0,0,0
104728_cam-np3-1-5,A4.1,0.007478,0,0,3,29,0,0,0
104728_cam-np3-2-10,A4.1,0.007234,0,0,26,0,0,0,0
104728_cam-np3-2-8,A4.7,0.008970,0,0,0,57,0,0,0
104728_cam-np3-2-9,A4.1,0.007015,0,0,26,52,0,0,0


In [196]:
sss_df.to_csv(os.path.join(new_folder,'sss_class.csv'))

In [55]:
len(sss_df)

420

In [56]:
os.mkdir(os.path.join(new_folder,'labeled_sss'))

In [119]:
from shutil import copyfile
from re import search
def getImagesPath(parent_path, filenames):
    not_found = []
    for f in filenames:
        folder = f[:search('-\d',f).start()]
        filepath = os.path.join(parent_path, folder, f + '.png')
        dstpath = os.path.join(parent_path, 'labeled_sss', f + '.png')
        if os.path.exists(filepath):
            copyfile(filepath, dstpath)
        else:
            not_found.append(f)
    return not_found

In [131]:
not_found = getImagesPath(new_folder,sss_df.index)
sss_df = sss_df.drop(not_found)

In [132]:
len(pd.unique(sss_df.index))

411

In [134]:
def createHash(img, bot, top):
    hashmap = {}
    for i,j in it.product(range(img.shape[0]), range(img.shape[1])):
        if img[i,j] not in hashmap:
            if img[i,j] <= bot:
                hashmap[img[i,j]] = 0
            elif img[i,j] > top:
                hashmap[img[i,j]] = 255
            else:
                hashmap[img[i,j]] = 255 * ((img[i,j] - bot) / (top - bot))
                
    return hashmap

In [194]:
def linearStretch(img):
    bot = min(img.ravel())
    top = max(img.ravel())
        
    hashmap = createHash(img, bot, top)
    img2 = img
    for i, j in it.product(range(img2.shape[0]), range(img2.shape[1])):
        img2[i,j] = hashmap[img2[i,j]]
        
    return img2

def powerStretch(img):
    img2 = img
    #img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2RGB)  
    img2 = img2/255.0      
    im_power_law_transformation = cv2.pow(img2,0.65)
    im_power_law_transformation *= 255 
    im_power_law_transformation = im_power_law_transformation.astype('uint8')
    
    return im_power_law_transformation

def integratedColorModel(img):
    hsvimg = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)   
    hue = hsvimg[:,:,0]
    saturation = hsvimg[:,:,1]
    value = hsvimg[:,:,2]
    valuecs = powerStretch(value)
    saturationcs = powerStretch(saturation)
    hlscs = cv2.merge((hue,saturationcs,valuecs))
    imc = cv2.cvtColor(hlscs, cv2.COLOR_HLS2RGB)
   
    return imc

def createImages(t):
    img = cv2.imread(os.path.join(cv_path,t))
    print('Processing', t)
    icm = integratedColorModel(img)
    cv2.imwrite(os.path.join(cv_path, icm_path,t), icm)

In [171]:
cv_path = os.path.join(new_folder,'labeled_sss')
icm_path = 'icm/'
if not os.path.exists(os.path.join(cv_path,icm_path)):
    os.mkdir(os.path.join(cv_path,icm_path))


In [195]:
pool = mp.Pool(processes=mp.cpu_count())
pool.map(createImages, [f for f in os.listdir(cv_path) if os.path.isfile(os.path.join(cv_path, f))])

Processing 132143_forcadinho-np3-2-9.png
Processing 125355_forcadinho-np3-8-10.png
Processing 112045_mb_survey-22-7.png
Processing 132143_forcadinho-np3-7-5.png
Processing 124011_mb_survey2-4-0.png
Processing 132614_mb_survey3-13-4.png
Processing 132143_forcadinho-np3-9-2.png
Processing 132143_forcadinho-np3-19-1.png
Processing 105317_cam-np3-6-1.png
Processing 125355_forcadinho-np3-2-10.png
Processing 105317_cam-np3-9-2.png
Processing 132143_forcadinho-np3-4-2.png
Processing 125355_forcadinho-np3-12-1.png
Processing 105317_cam-np3-1-7.png
Processing 132614_mb_survey3-20-6.png
Processing 125355_forcadinho-np3-11-5.png
Processing 143418_mb_survey4-13-2.png
Processing 105317_cam-np3-12-5.png
Processing 132143_forcadinho-np3-31-6.png
Processing 132143_forcadinho-np3-19-0.png
Processing 113610_cam_survey_1-3-5.png
Processing 125355_forcadinho-np3-3-1.png
Processing 112045_mb_survey-23-2.png
Processing 105317_cam-np3-7-2.png
Processing 132143_forcadinho-np3-34-9.png
Processing 132143_forcad

Processing 132143_forcadinho-np3-35-9.png
Processing 132143_forcadinho-np3-21-5.png
Processing 132143_forcadinho-np3-38-6.png
Processing 132614_mb_survey3-26-10.png
Processing 105317_cam-np3-12-9.png
Processing 124011_mb_survey2-2-4.png
Processing 125355_forcadinho-np3-10-6.png
Processing 125355_forcadinho-np3-9-2.png
Processing 112045_mb_survey-2-2.png
Processing 132614_mb_survey3-1-3.png
Processing 132143_forcadinho-np3-17-3.png
Processing 132143_forcadinho-np3-28-5.png
Processing 105317_cam-np3-9-1.png
Processing 125355_forcadinho-np3-3-9.png
Processing 132143_forcadinho-np3-33-5.png
Processing 105317_cam-np3-3-2.png
Processing 112045_mb_survey-4-8.png
Processing 132143_forcadinho-np3-6-8.png
Processing 132143_forcadinho-np3-36-1.png
Processing 132143_forcadinho-np3-30-5.png
Processing 105317_cam-np3-12-1.png
Processing 112045_mb_survey-19-8.png
Processing 124011_mb_survey2-21-11.png
Processing 132143_forcadinho-np3-7-10.png
Processing 105317_cam-np3-6-10.png
Processing 132143_forca

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [197]:
from sklearn.model_selection import train_test_split
from IPython.display import SVG,display
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from keras.applications import VGG16
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing.image import load_img
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

/home/diegues/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [198]:
def createDirectoryEnvironment(train_data, train_targets, val_data, val_targets, path):
    trainpath = path + 'train/'
    valpath = path + 'validation/'
    if(os.listdir() == 2):
        return trainpath,valpath
    if(not os.path.exists(trainpath)):
        os.mkdir(trainpath)
    if(not os.path.exists(valpath)):
        os.mkdir(valpath)
        
    classes = train_targets.columns.append(val_targets.columns).drop_duplicates()  
    for c in classes:
        train_class = trainpath + c + '/'
        val_class = valpath + c + '/'
        if(not os.path.exists(train_class)):
            os.mkdir(train_class)
        if(not os.path.exists(val_class)):
            os.mkdir(val_class)
            
    for f in train_data.index.values:
        copyfile(path + '../CV/' + f[:f.find('frame')-1] + '/icm/' + f, trainpath + train_targets.loc[f].idxmax(axis = 1) + '/' + f)
    for f in val_data.index.values:
        copyfile(path + '../CV/' + f[:f.find('frame')-1] + '/icm/' + f, valpath + val_targets.loc[f].idxmax(axis = 1) + '/' + f)
    
    return trainpath, valpath

In [ ]:
folder_path = new_folder
data = pd.read_csv(folder_path + "sss_class.csv")

filenames = data['sss_name']
targets = data['level3']

# one-hot encoding
targets_ohe = pd.get_dummies(targets)

# dealing with NaNs
data = data.drop(['roll', 'pitch', 'level1', 'level2', 'level3', 'level4', 
                  'level5', 'level6', 'AphiaID', 'EunisName', 'EunisCode', 
                  'date', 'timestamp', 'species'],
                 axis = 1)

X = data.groupby('filename').max()
Y = pd.concat([filenames,targets_ohe], axis = 1).groupby('filename').max()


# tts
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size = 0.3, random_state=47)

images_path = '/home/diegues/Desktop/ProcessedImages/SampledData/'
train_dir, val_dir = createDirectoryEnvironment(train_X, train_Y, test_X, test_Y, images_path)

In [200]:
vgg16_ft = VGG16(weights = '/home/diegues/git/AutomaticHabitatMapping/src/weights/eunis_photo_classifier.h5')
vgg16_ft.summary()

ResourceExhaustedError: OOM when allocating tensor with shape[25088,4096] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[Node: fc1/kernel/Assign = Assign[T=DT_FLOAT, use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](fc1/kernel, fc1/random_uniform)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'fc1/kernel/Assign', defined at:
  File "/home/diegues/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/diegues/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/diegues/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/diegues/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in la